In [1]:
import numpy as np
import scipy as sp
from scipy import io,integrate,sparse
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.lines import Line2D
import matplotlib.patches as mpatches
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

from kpm_bin import *

from IPython.display import clear_output
np.set_printoptions(linewidth=300)
%load_ext autoreload

%autoreload 2

## TeX preamble
preamble = r"""
\usepackage[no-math]{fontspec}
\setmainfont[]{Helvetica Condensed}

\usepackage{amsmath,amssymb,amsthm}
\usepackage{mathastext}
"""

mpl.use("pgf")

mpl.rcParams.update({
    "pgf.texsystem": "lualatex",
    'font.family': 'serif',
    'font.size' : 8,
    'text.usetex': True,
    'pgf.rcfonts': False,
    'pgf.preamble': preamble,
})

In [2]:
N = 20
s = 1/2
M = int(2*s+1)

d = M**N

J_t = np.zeros((N,N))
for i in range(N):
    for j in range(N):
        if np.abs(i-j)==1:
            J_t[i,j] = 1
            
J = 1
h = 12

In [3]:
H = get_hamiltonian((J/4)*J_t,(J/4)*J_t,0*J_t,h/2,s)

In [4]:
Evs = get_solvable_density_EVs(h,J,N)

In [5]:
Emin_true = Evs[0]
Emax_true = Evs[-1]

In [6]:
m = 1
k = 250
np.random.seed(0)

αβs = []
for _ in range(m):
    print(f'random sample {_}')
    clear_output(wait=True)

    v = np.random.randn(d)
    v /= np.linalg.norm(v)

    (α,β) = lanczos(H,v,k)
    αβs.append((α,β))

random sample 0


In [7]:
k_eff = min(500,2*k)

ϵs = [-7e-4,.5,0]

ϵ = .015
Emin = Emin_true - ϵ*(Emax_true - Emin_true)
Emax = Emax_true + ϵ*(Emax_true - Emin_true)

EE = np.linspace(Emin,Emax,1000)


fig,axs = plt.subplots(1,1,figsize=(12,3))
plt.subplots_adjust(bottom=.18,top=.9)

for i,ϵ in enumerate(ϵs):
    Emin = Emin_true - ϵ*(Emax_true - Emin_true)
    Emax = Emax_true + ϵ*(Emax_true - Emin_true)

    μs = []
    for (α,β) in αβs:

        σ = get_chebT_density(Emin,Emax)
        (γ,δ) = get_chebT_recurrence(2*k,Emin,Emax)

        Hk = np.diag(β,1) + np.diag(β,-1) + np.diag(np.append(α,0))
        e0 = np.zeros(k+1)
        e0[0] = 1
    
        μ = get_moments(Hk,e0,2*k,γ,δ)

        μs.append(μ)


    # for μ in μs:
    #     dρdσ = get_op_expansion(μ,γ,δ)
    #     ρ_KPM = lambda E: σ(E)*dρdσ(E)

    #     plt.plot(EE,ρ_KPM(EE),lw=.5,c='black',alpha=.2)

   
    g=jackson_weights(k_eff)
    
    dρdσ = get_op_expansion(g*np.mean(μs,axis=0)[:k_eff],γ,δ)
    ρ_KPM = lambda E: σ(E)*dρdσ(E)

    styles = [{'color':plt.cm.magma(.85),'lw':1,'alpha':1},\
              {'color':plt.cm.magma(.5),'lw':1,'alpha':1},\
              {'color':plt.cm.magma(.12),'lw':1,'alpha':1}]
    
    plt.plot(EE,ρ_KPM(EE),**styles[i],label=rf'$\epsilon={ϵ}$')

plt.hist(Evs,300,density=True,histtype='stepfilled',lw=.5,ec='black',fc='#dddddd',label='true DOS');

plt.xlabel(r'energy \emph{E}')
plt.ylabel('density of states')

#plt.legend(loc='upper right')
#plt.yscale('log')
plt.ylim(-.001,.03)

plt.savefig('imgs/support.pdf')